In [1]:
# doing some imports
import json
import pickle
from haystack.document_store.elasticsearch import ElasticsearchDocumentStore

/home/rich/anaconda3/envs/ml/lib/python3.8/site-packages/ray/autoscaler/_private/cli_logger.py:57: FutureWarning: Not all Ray CLI dependencies were found. In Ray 1.4+, the Ray CLI, autoscaler, and dashboard will only be usable via `pip install 'ray[default]'`. Please update your install command.
  warnings.warn(


In [2]:
# bringing in the list of the answers and the 
# the questions and the context from the squad list.
#
with open("pred_ans_list", mode="rb") as f:
    pred_ans_list = pickle.load(f,)


with open("true_ans_list", mode="rb") as f:
    true_ans_list = pickle.load(f)

with open("questions_list", mode="rb") as f:
    questions_list = pickle.load(f)

with open("contexts", mode="rb") as f:
    contexts = pickle.load(f)

In [5]:
print(f"The length of the pred_ans_list is: {len(pred_ans_list)}")
print(f"This is what the pred ans list looks like: {pred_ans_list[:3]}")

print(f"The length of the contexts that is brought in is {len(contexts)}")

The length of the pred_ans_list is: 10000
This is what the pred ans list looks like: ['late 1990s', 'singing and dancing', '2003']
The length of the contexts that is brought in is 130319


In [4]:
print(f"The length of the true_ans_list is: {len(true_ans_list)}")
true_ans_list[:3]

The length of the true_ans_list is: 10000


['in the late 1990s', 'singing and dancing', '2003']

In [6]:
with open("squad_ans", mode="rb") as f:
    squad_ans = pickle.load(f)

In [9]:
squad_ans.keys()

dict_keys(['pred_ans', 'questions', 'true_ans'])

In [10]:
# checking the size of the things that are in the squad_ans dictionary
print(len(squad_ans["pred_ans"]))
print(len(squad_ans["questions"]))
print(len(squad_ans["true_ans"]))

10000
10000
10000


In [3]:
the_doc_store = ElasticsearchDocumentStore(
    host = "localhost", username="", 
    password="", index="squad_docs"
)


/home/rich/anaconda3/envs/ml/lib/python3.8/site-packages/elasticsearch/connection/base.py:190: ElasticsearchDeprecationWarning: Elasticsearch built-in security features are not enabled. Without authentication, your cluster could be accessible to anyone. See https://www.elastic.co/guide/en/elasticsearch/reference/7.15/security-minimal-setup.html to enable security.
  warnings.warn(message, category=ElasticsearchDeprecationWarning)
11/30/2021 11:57:43 - INFO - elasticsearch -   HEAD http://localhost:9200/ [status:200 request:0.575s]
11/30/2021 11:57:43 - INFO - elasticsearch -   HEAD http://localhost:9200/squad_docs [status:200 request:0.015s]
11/30/2021 11:57:43 - INFO - elasticsearch -   GET http://localhost:9200/squad_docs [status:200 request:0.014s]
11/30/2021 11:57:43 - INFO - elasticsearch -   PUT http://localhost:9200/squad_docs/_mapping [status:200 request:0.028s]
11/30/2021 11:57:43 - INFO - elasticsearch -   HEAD http://localhost:9200/label [status:200 request:0.003s]


In [4]:
# want to see if the host is found
import requests

In [5]:
res = requests.get("http://localhost:9200/_cluster/health")

In [6]:
res.json()

{'cluster_name': 'elasticsearch',
 'status': 'yellow',
 'timed_out': False,
 'number_of_nodes': 1,
 'number_of_data_nodes': 1,
 'active_primary_shards': 3,
 'active_shards': 3,
 'relocating_shards': 0,
 'initializing_shards': 0,
 'unassigned_shards': 2,
 'delayed_unassigned_shards': 0,
 'number_of_pending_tasks': 0,
 'number_of_in_flight_fetch': 0,
 'task_max_waiting_in_queue_millis': 0,
 'active_shards_percent_as_number': 60.0}

In [10]:
# the data for the elastic search 
# which is as follows
# as a dictionary 
# { "text": (here is the context of the squad data)
# }
#
the_context_data = []
for count , each_context in enumerate(contexts):
    the_context_data.append({"text": each_context, 
                            "meta": {"count":count}
                            })
# looking at what they look like
the_context_data[:2]

[{'text': 'Beyoncé Giselle Knowles-Carter (/biːˈjɒnseɪ/ bee-YON-say) (born September 4, 1981) is an American singer, songwriter, record producer and actress. Born and raised in Houston, Texas, she performed in various singing and dancing competitions as a child, and rose to fame in the late 1990s as lead singer of R&B girl-group Destiny\'s Child. Managed by her father, Mathew Knowles, the group became one of the world\'s best-selling girl groups of all time. Their hiatus saw the release of Beyoncé\'s debut album, Dangerously in Love (2003), which established her as a solo artist worldwide, earned five Grammy Awards and featured the Billboard Hot 100 number-one singles "Crazy in Love" and "Baby Boy".',
  'meta': {'count': 0}},
 {'text': 'Beyoncé Giselle Knowles-Carter (/biːˈjɒnseɪ/ bee-YON-say) (born September 4, 1981) is an American singer, songwriter, record producer and actress. Born and raised in Houston, Texas, she performed in various singing and dancing competitions as a child, a

In [8]:
# showing the lenght of the context data
print(f"the length of the context_data is: {len(the_context_data)}")

the length of the context_data is: 130319


In [11]:
the_doc_store.write_documents(the_context_data)

/home/rich/anaconda3/envs/ml/lib/python3.8/site-packages/elasticsearch/connection/base.py:190: ElasticsearchDeprecationWarning: Elasticsearch built-in security features are not enabled. Without authentication, your cluster could be accessible to anyone. See https://www.elastic.co/guide/en/elasticsearch/reference/7.15/security-minimal-setup.html to enable security.
  warnings.warn(message, category=ElasticsearchDeprecationWarning)
11/29/2021 13:29:11 - INFO - elasticsearch -   POST http://localhost:9200/_bulk?refresh=wait_for [status:200 request:4.137s]
11/29/2021 13:29:12 - INFO - elasticsearch -   POST http://localhost:9200/_bulk?refresh=wait_for [status:200 request:1.151s]
11/29/2021 13:29:13 - INFO - elasticsearch -   POST http://localhost:9200/_bulk?refresh=wait_for [status:200 request:1.135s]
11/29/2021 13:29:14 - INFO - elasticsearch -   POST http://localhost:9200/_bulk?refresh=wait_for [status:200 request:1.138s]
11/29/2021 13:29:15 - INFO - elasticsearch -   POST http://localho

Querying the data (GETTING THE DATA)

In [12]:
from haystack.retriever.sparse import TfidfRetriever

In [13]:
retriever = TfidfRetriever(the_doc_store)

11/29/2021 13:34:41 - INFO - elasticsearch -   POST http://localhost:9200/squad_docs/_search?scroll=1d&size=10000 [status:200 request:3.094s]
11/29/2021 13:34:42 - INFO - elasticsearch -   POST http://localhost:9200/_search/scroll [status:200 request:0.912s]
11/29/2021 13:34:46 - INFO - elasticsearch -   POST http://localhost:9200/_search/scroll [status:200 request:0.068s]
11/29/2021 13:34:46 - INFO - elasticsearch -   DELETE http://localhost:9200/_search/scroll [status:200 request:0.050s]
11/29/2021 13:34:46 - INFO - haystack.retriever.sparse -   Found 19029 candidate paragraphs from 19029 docs in DB


In [22]:
len(the_context_data)

130319

In [14]:
# Trying a query
query = "Beyonce"

In [15]:
the_docs = retriever.retrieve(query=query)

In [16]:
len(the_docs)

10

In [17]:
the_docs

[{'text': 'In August, the couple attended the 2011 MTV Video Music Awards, at which Beyoncé performed "Love on Top" and started the performance saying "Tonight I want you to stand up on your feet, I want you to feel the love that\'s growing inside of me". At the end of the performance, she dropped her microphone, unbuttoned her blazer and rubbed her stomach, confirming her pregnancy she had alluded to earlier in the evening. Her appearance helped that year\'s MTV Video Music Awards become the most-watched broadcast in MTV history, pulling in 12.4 million viewers; the announcement was listed in Guinness World Records for "most tweets per second recorded for a single event" on Twitter, receiving 8,868 tweets per second and "Beyonce pregnant" was the most Googled term the week of August 29, 2011.', 'score': None, 'question': None, 'meta': {'count': 347}, 'embedding': None, 'id': '41736eaf87e42d59b7a25a873f730daf'},
 {'text': 'In September 2010, West wrote a series of apologetic tweets add

# Cleaning the index

In [21]:
# will first remove all the index from the elasticsearch
# will do this using the requests
# checking the amount in the index at the moment
res = requests.post("http://localhost:9200/squad_docs/_delete_by_query", 
json={
    "query": {
        "match_all": {}
    }
})
res = requests.get("http://localhost:9200/squad_docs/_count")
res.json()

{'count': 0,
 '_shards': {'total': 1, 'successful': 1, 'skipped': 0, 'failed': 0}}

In [5]:
# will now make the context with no duplicates
# using a set
m_set = set(contexts)
# looping through the context to make just one copy of the contexts
contexts  = list(m_set)
print(f"The length of the contexts list is now {len(contexts)}")

# now making the data in the correct format to be used by elasticsearch
doc_to_add = [{"text": c} for c in contexts]
len(doc_to_add)

The length of the contexts list is now 19029


19029

In [6]:
the_doc_store.write_documents(doc_to_add)

11/30/2021 11:58:15 - INFO - elasticsearch -   POST http://localhost:9200/_bulk?refresh=wait_for [status:200 request:0.954s]
11/30/2021 11:58:16 - INFO - elasticsearch -   POST http://localhost:9200/_bulk?refresh=wait_for [status:200 request:1.116s]
11/30/2021 11:58:17 - INFO - elasticsearch -   POST http://localhost:9200/_bulk?refresh=wait_for [status:200 request:1.091s]
11/30/2021 11:58:18 - INFO - elasticsearch -   POST http://localhost:9200/_bulk?refresh=wait_for [status:200 request:1.075s]
11/30/2021 11:58:19 - INFO - elasticsearch -   POST http://localhost:9200/_bulk?refresh=wait_for [status:200 request:1.068s]
11/30/2021 11:58:20 - INFO - elasticsearch -   POST http://localhost:9200/_bulk?refresh=wait_for [status:200 request:1.059s]
11/30/2021 11:58:21 - INFO - elasticsearch -   POST http://localhost:9200/_bulk?refresh=wait_for [status:200 request:1.078s]
11/30/2021 11:58:22 - INFO - elasticsearch -   POST http://localhost:9200/_bulk?refresh=wait_for [status:200 request:1.070s]


# Using the BM25 retriever

In [7]:
from haystack.retriever import ElasticsearchRetriever

In [8]:
retriever = ElasticsearchRetriever(the_doc_store)

In [9]:
# now will try the query
q = "Who made the ipod?"

In [11]:
retrieved_list = retriever.retrieve(q)

/home/rich/anaconda3/envs/ml/lib/python3.8/site-packages/elasticsearch/connection/base.py:190: ElasticsearchDeprecationWarning: Elasticsearch built-in security features are not enabled. Without authentication, your cluster could be accessible to anyone. See https://www.elastic.co/guide/en/elasticsearch/reference/7.15/security-minimal-setup.html to enable security.
  warnings.warn(message, category=ElasticsearchDeprecationWarning)
11/30/2021 12:09:41 - INFO - elasticsearch -   POST http://localhost:9200/squad_docs/_search [status:200 request:0.099s]


In [13]:
retrieved_list

[{'text': 'Many accessories have been made for the iPod line. A large number are made by third party companies, although many, such as the iPod Hi-Fi, are made by Apple. Some accessories add extra features that other music players have, such as sound recorders, FM radio tuners, wired remote controls, and audio/visual cables for TV connections. Other accessories offer unique features like the Nike+iPod pedometer and the iPod Camera Connector. Other notable accessories include external speakers, wireless remote controls, protective case, screen films, and wireless earphones. Among the first accessory manufacturers were Griffin Technology, Belkin, JBL, Bose, Monster Cable, and SendStation.', 'score': 0.858503345068895, 'question': None, 'meta': {}, 'embedding': None, 'id': '4b8ed923c756c2267f28017348d95b04'},
 {'text': 'On April 9, 2007, it was announced that Apple had sold its one-hundred millionth iPod, making it the biggest selling digital music player of all time. In April 2007, Apple